# Data Loader

#### **Setup di PRAW, Datetime e Pandas**: In questa cella, vengono importate le librerie necessarie (PRAW per l'accesso a Reddit, datetime per la gestione delle date e Pandas per la manipolazione dei dati tabellari).

In [232]:
import praw
from datetime import datetime
import pandas as pd

#### Configurazione dell'istanza di reddit

In [233]:
reddit = praw.Reddit(
    client_id="#",
    client_secret="#",
    #password="PASSWORD",
    user_agent="#",
    #username="USERNAME",
)


#### **Configurazione del Subreddit e Recupero dei Top Posts**: In questa cella, viene specificato il subreddit da cui vogliamo recuperare i post ("Italia"), quindi viene eseguita una ricerca per i post contenenti "giulia cecchettin", ordinati per popolarità (top) e considerando tutti i tempi (time_filter='all'). I risultati selezionati sono 20 post.

In [234]:
subreddit = reddit.subreddit("Italia")
top_posts_giulia_cecchettin = subreddit.search('giulia cecchettin', sort='top', time_filter='all',limit=20)
top_posts_giulia_tramontano = subreddit.search('impagnatiello', sort='top', time_filter='all',limit=20)

#### **Inizializzazione del DataFrame**: Creazione di un DataFrame vuoto con Pandas utilizzato per immagazzinare i commenti estratti.

In [235]:
df_cecchettin = pd.DataFrame()
df_tramontano = pd.DataFrame()

#### **Funzione di Conversione della Data**: Definizione di una funzione `converti` che converte l'epoch timestamp in un formato di data leggibile.

In [236]:
def converti(data):
    data_leggibile = datetime.utcfromtimestamp(data).strftime('%Y-%m-%d')
    #print(data_leggibile)
    return data_leggibile

#### **Estrazione dei Commenti e Aggiunta al DataFrame**: Attraverso un ciclo `for` si itera sui post principali e i relativi commenti. Viene estratto il testo del commento e la data di creazione, per poi essere aggiunta al DataFrame.

##### Giulia Cecchettin

In [237]:
df_cecchettin = pd.DataFrame(columns=['Comment', 'Date'])

def process_comment(comment):
    try:
        print(comment.body)
        df_cecchettin.loc[len(df_cecchettin)] = [comment.body, converti(comment.created_utc)]
    except:
        pass
    
    try:
        list_replies = comment.replies.list()
        for reply_comment in list_replies:
            process_comment(reply_comment)
    except:
        pass

for post in top_posts_giulia_cecchettin:
    print("Title: ", post.title)
    for comment in post.comments:
        process_comment(comment)
    print("-----------------------------------------")


##### Giulia Tramontano

In [238]:
df_tramontano = pd.DataFrame(columns=['Comment', 'Date'])

def process_comment(comment):
    try:
        print(comment.body)
        df_tramontano.loc[len(df_tramontano)] = [comment.body, converti(comment.created_utc)]
    except:
        pass
    
    try:
        list_replies = comment.replies.list()
        for reply_comment in list_replies:
            process_comment(reply_comment)
    except:
        pass

for post in top_posts_giulia_tramontano:
    print("Title: ", post.title)
    for comment in post.comments:
        process_comment(comment)
    print("-----------------------------------------")


#### Pre-processing dei dati prima di effettuare il salvataggio.

##### Giulia Cecchettin

In [ ]:
i = 31
while(i != 4):
    date = "2023-12-" + str(i)
    df_cecchettin.drop(df_cecchettin[df_cecchettin['Date'] == date].index, inplace=True)
    i -= 1

i = 31
while(i != 0):
    date = "2024-01-" + str(i)
    df_cecchettin.drop(df_cecchettin[df_cecchettin['Date'] == date].index, inplace=True)
    i -= 1

df_cecchettin.sort_values(by='Date', inplace=True)
df_cecchettin = df_cecchettin.loc[~df_cecchettin['Comment'].str.contains(r'\[removed\]|\[deleted\]')]
df_cecchettin.drop_duplicates(inplace=True)
df_cecchettin.reset_index(drop=True, inplace=True)

##### Giulia Tramontano

In [ ]:
df_tramontano.sort_values(by='Date', inplace=True)
df_tramontano = df_tramontano.loc[~df_tramontano['Comment'].str.contains(r'\[removed\]|\[deleted\]')]
df_tramontano.drop_duplicates(inplace=True)
df_tramontano.reset_index(drop=True, inplace=True)

#### **Salvataggio del DataFrame in un File CSV**: Si converte il DataFrame in un file CSV.

##### Giulia Cecchettin

In [239]:
df_cecchettin.to_csv("../dataset_input/Giulia_cecchettin.csv", index=False)


In [240]:
pd.read_csv('../dataset_input/Giulia_cecchettin.csv')

##### Giulia Tramontano

In [241]:
df_tramontano.to_csv("../dataset_input/Giulia_Tramontano", index=False)


In [242]:
pd.read_csv('../dataset_input/Giulia_Tramontano.csv')